In [1]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
import os
# from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
saved_model = 'saved_model'
labels = 'label_map.pbtxt'
annotations = 'annotations'

saved_model_path = os.path.abspath(os.path.join('.', saved_model))
labels_path = os.path.abspath(os.path.join('.', annotations, labels))

print(f"\nsaved_model_path: {saved_model_path}")
print(f"\nlabels_path: {labels_path}")
    
# PROVIDE PATH TO MODEL DIRECTORY
# PATH_TO_MODEL_DIR = 'E:\\DataSc\\Practicals\\2.Object_Detection\\Downloaded_From_Colab\\05.29.2021_Traffic\\trained_model\\content\\driving-object-detection\\exported-models\\saved_model'

# # PROVIDE PATH TO LABEL MAP
# PATH_TO_LABELS = 'E:\\DataSc\\Practicals\\2.Object_Detection\\Post_Training\\annotations\\label_map.pbtxt'

# PATH_TO_SAVED_MODEL ="E:\\DataSc\\Practicals\\2.Object_Detection\\Post_Training\\saved_model"


saved_model_path: E:\DataSc\Practicals\2.Object_Detection\Post_Training_light\saved_model

labels_path: E:\DataSc\Practicals\2.Object_Detection\Post_Training_light\annotations\label_map.pbtxt


In [7]:
# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.30)

# LOAD THE MODEL
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

print('Loading model...', end='')
start_time = time.time()


# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(saved_model_path)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING
category_index = label_map_util.create_category_index_from_labelmap(labels_path, use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

Loading model...Done! Took 116.80568099021912 seconds


In [15]:
image_name = 'green_51.jpg'     #Succeeded
# image_name = 'red_448.png'      #Failed
# image_name = 'red_545.jpg'      #Failed
# image_name = 'red_656.png'      #Failed
# image_name = 'yellow_858.jpg'   #Succeeded

images = 'images'

image_path = os.path.abspath(os.path.join('.', images, image_name))

print('Running inference for {}... '.format(image_path), end='')

image = cv2.imread(image_path)

print(f"image_path: {image_path}")
print(type(image))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)

# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}

detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.2,
      agnostic_mode=False)

print('Done 1')
# DISPLAYS OUTPUT IMAGE

%matplotlib inline
plt.figure(figsize=(150,150))

plt.imshow(image_with_detections)
print('Done 2')
plt.show()

# CLOSES WINDOW ONCE KEY IS PRESSED

Running inference for E:\DataSc\Practicals\2.Object_Detection\Post_Training_light\images\green_51.jpg... image_path: E:\DataSc\Practicals\2.Object_Detection\Post_Training_light\images\green_51.jpg
<class 'numpy.ndarray'>
Done 1
Done 2
